NOTA: Es importante instalar versiones de libreria **osmnx == 1.1** y **matplotlib == 3.5**

In [ ]:
# Librerias

## NOTA: Usar version de libreria osmnx==1.1 y matplotlib==3.5

import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import geocoders
from geopy.distance import geodesic
from shapely.geometry import LineString, Point
import folium
import osmnx as ox
import networkx as nx
import sklearn

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# leer datos de direccion por punto

df = [['221 4th St, San Francisco, CA 94103','3223 Ortega St, San    Francisco, CA 94122'],
         ['2025 Hamilton Ave, San Jose, CA 95125','488 S Almaden Ave, San Jose, CA 95110'],
         ['3720 Stephen M White Dr, San Pedro, CA 90731','500 Sea World Dr, San Diego, CA 92109'],
         ['1010 Pearl St #7B, La Jolla, CA 92037','19000 NE Sandy Blvd, Portland, OR 97230']]
df = pd.DataFrame(df, columns = ['Address1', 'Address2'])
df

,Address1,Address2
0,"221 4th St, San Francisco, CA 94103","3223 Ortega St, San Francisco, CA 94122"
1,"2025 Hamilton Ave, San Jose, CA 95125","488 S Almaden Ave, San Jose, CA 95110"
2,"3720 Stephen M White Dr, San Pedro, CA 90731","500 Sea World Dr, San Diego, CA 92109"
3,"1010 Pearl St #7B, La Jolla, CA 92037","19000 NE Sandy Blvd, Portland, OR 97230"


In [ ]:
# calcular o buscar coordenadas espaciales (latitud,longitud)

lista_agentes = ["PDS","GetLoc","GeoNames"]
i_agente = 0

#geolocator = Nominatim(timeout=100, user_agent="PDS")
#geolocator = Nominatim(timeout=100, user_agent="GetLoc")
#geolocator = Nominatim(timeout=100, user_agent="GeoNames")
geolocator =  Nominatim(timeout=100, user_agent=lista_agentes[i_agente])
print('Agente seleccionado:',lista_agentes[i_agente])


df["cord1"] = df["Address1"].apply(geolocator.geocode)
df['lat1'] = [g.latitude if g else g for g in df["cord1"]]
df['lon1'] = [g.longitude if g else g for g in df["cord1"]]

df["cord2"] = df["Address2"].apply(geolocator.geocode)
df['lat2'] = [g.latitude if g else g for g in df["cord2"]]
df['lon2'] = [g.longitude if g else g for g in df["cord2"]]
df

Agente seleccionado: PDS


,Address1,Address2,cord1,lat1,lon1,cord2,lat2,lon2
0,"221 4th St, San Francisco, CA 94103","3223 Ortega St, San Francisco, CA 94122","(221, 4th Street, South of Market, San Francis...",37.783101,-122.402363,"(3223, Ortega Street, Sunset District, San Fra...",37.751351,-122.497609
1,"2025 Hamilton Ave, San Jose, CA 95125","488 S Almaden Ave, San Jose, CA 95110","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,"(488, South Almaden Avenue, SoFA, San Jose, Sa...",37.327363,-121.888325
2,"3720 Stephen M White Dr, San Pedro, CA 90731","500 Sea World Dr, San Diego, CA 92109",None,NaN,NaN,"(SeaWorld San Diego, 500, Sea World Drive, San...",32.765978,-117.226626
3,"1010 Pearl St #7B, La Jolla, CA 92037","19000 NE Sandy Blvd, Portland, OR 97230",None,NaN,NaN,None,NaN,NaN


In [ ]:
# calcular distancia entre direccion 1 y direccion 2

def distancia_geo(row): 
    add1 = (row["lat1"], row["lon1"]) 
    add2 = (row["lat2"], row["lon2"]) 
    try: 
        return (geodesic(add1, add2).km) 
    except:
        return np.nan

df["distancia_km"] = df.apply(lambda row: distancia_geo(row), axis=1).round(3)
df

,Address1,Address2,cord1,lat1,lon1,cord2,lat2,lon2,distancia_km
0,"221 4th St, San Francisco, CA 94103","3223 Ortega St, San Francisco, CA 94122","(221, 4th Street, South of Market, San Francis...",37.783101,-122.402363,"(3223, Ortega Street, Sunset District, San Fra...",37.751351,-122.497609,9.102
1,"2025 Hamilton Ave, San Jose, CA 95125","488 S Almaden Ave, San Jose, CA 95110","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,"(488, South Almaden Avenue, SoFA, San Jose, Sa...",37.327363,-121.888325,4.141
2,"3720 Stephen M White Dr, San Pedro, CA 90731","500 Sea World Dr, San Diego, CA 92109",None,NaN,NaN,"(SeaWorld San Diego, 500, Sea World Drive, San...",32.765978,-117.226626,NaN
3,"1010 Pearl St #7B, La Jolla, CA 92037","19000 NE Sandy Blvd, Portland, OR 97230",None,NaN,NaN,None,NaN,NaN,NaN


In [ ]:
# crear grafica con puntos y rutas

centro_lat = np.nanmean(pd.concat([df.lat1,df.lat2]).tolist())
centro_lon = np.nanmean(pd.concat([df.lon1,df.lon2]).tolist())

mapa = folium.Map(location=(centro_lat,centro_lon), zoom_start=5)
for index,row in df.iterrows():
    
    points = []
    
    if((row["lat1"] == row["lat1"]) & (row["lon1"] == row["lon1"])):
        points.append((row["lat1"],row["lon1"]))
        folium.Marker(location=(row['lat1'], 
                                row['lon1']),
                      popup=row["Address1"],
                      ).add_to(mapa)
    
    if((row["lat2"] == row["lat2"]) & (row["lon2"] == row["lon2"])):
        points.append((row["lat2"],row["lon2"]))
        folium.Marker(location=(row['lat2'], 
                                row['lon2']),
                      popup=row["Address2"],
                      ).add_to(mapa)
        
    if(len(points)>0):
        folium.PolyLine(points
                        ,color="blue"
                        ,weight=2.5
                        ,popup= str(row["distancia_km"])+ " km"
                        ,opacity=1).add_to(mapa)

display(mapa)

Parte 2 - Calculo de distancias a 1 solo punto

In [ ]:
# crear backup
df_backup = df.copy()

In [ ]:
# Obtener lugar con coordenadas (no considerar aquellos sin posicion)

lugares = dict()
for i in range(df.shape[0]):
    dirp = df.iloc[i,:][["cord1","lat1","lon1"]] 
    if((dirp == dirp).all()):
        lugares[df.iloc[i,:]["Address1"]] = dirp
        
    dirp = df.iloc[i,:][["cord2","lat2","lon2"]] 
    if((dirp == dirp).all()):
        lugares[df.iloc[i,:]["Address2"]] = dirp
        
lugares

{'221 4th St, San Francisco, CA 94103': cord1    (221, 4th Street, South of Market, San Francis...
 lat1                                             37.783101
 lon1                                           -122.402363
 Name: 0, dtype: object,
 '3223 Ortega St, San    Francisco, CA 94122': cord2    (3223, Ortega Street, Sunset District, San Fra...
 lat2                                             37.751351
 lon2                                           -122.497609
 Name: 0, dtype: object,
 '2025 Hamilton Ave, San Jose, CA 95125': cord1    (Hamilton Avenue, Dry Creek Village, Willow Gl...
 lat1                                             37.292988
 lon1                                           -121.906481
 Name: 1, dtype: object,
 '488 S Almaden Ave, San Jose, CA 95110': cord2    (488, South Almaden Avenue, SoFA, San Jose, Sa...
 lat2                                             37.327363
 lon2                                           -121.888325
 Name: 1, dtype: object,
 '500 Sea Wor

In [ ]:
# Seleccionar aleatoriamente 1 punto

import random

# extraer aleatoriamente indice
ii = random.randint(0,len(lugares)-1)
print("indice:",ii)

# obtener valores de indice
nom_sel = list(lugares.keys())[ii]
dir_sel = lugares[nom_sel]

print(nom_sel)
print(dir_sel)

indice: 2
2025 Hamilton Ave, San Jose, CA 95125
cord1    (Hamilton Avenue, Dry Creek Village, Willow Gl...
lat1                                             37.292988
lon1                                           -121.906481
Name: 1, dtype: object


In [ ]:
# Asigna coordenadas y calcula distancias (km)

# Actualizar datos segun puntos con coordenadas y punto seleccionado (address 2)

df = pd.DataFrame()
ix = 0

for key,value in lugares.items():
    df.loc[ix,["address1"]] = key
    df.loc[ix,["cord1","lat1","lon1"]] = value.values
    
    df.loc[ix,["address2"]] = nom_sel
    df.loc[ix,["cord2","lat2","lon2"]] = dir_sel.values
    
    ix += 1


##########################
# calcular distancias entre puntos coordenados

def distancia_geo(row): 
    add1 = (row["lat1"], row["lon1"]) 
    add2 = (row["lat2"], row["lon2"]) 
    try: 
        return (geodesic(add1, add2).km) 
    except:
        return np.nan

df["distancia_km"] = df.apply(lambda row: distancia_geo(row), axis=1).round(3)
df

,address1,cord1,lat1,lon1,address2,cord2,lat2,lon2,distancia_km
0,"221 4th St, San Francisco, CA 94103","(221, 4th Street, South of Market, San Francis...",37.783101,-122.402363,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,69.855
1,"3223 Ortega St, San Francisco, CA 94122","(3223, Ortega Street, Sunset District, San Fra...",37.751351,-122.497609,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,72.929
2,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,0.000
3,"488 S Almaden Ave, San Jose, CA 95110","(488, South Almaden Avenue, SoFA, San Jose, Sa...",37.327363,-121.888325,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,4.141
4,"500 Sea World Dr, San Diego, CA 92109","(SeaWorld San Diego, 500, Sea World Drive, San...",32.765978,-117.226626,"2025 Hamilton Ave, San Jose, CA 95125","(Hamilton Avenue, Dry Creek Village, Willow Gl...",37.292988,-121.906481,659.053


In [ ]:
# crear grafica con puntos y rutas

centro_lat = np.nanmean(pd.concat([df.lat1,df.lat2]).tolist())
centro_lon = np.nanmean(pd.concat([df.lon1,df.lon2]).tolist())

mapa = folium.Map(location=(centro_lat,centro_lon), zoom_start=5)
for index,row in df.iterrows():
    
    points = []
    
    if((row["lat1"] == row["lat1"]) & (row["lon1"] == row["lon1"])):
        points.append((row["lat1"],row["lon1"]))
        folium.Marker(location=(row['lat1'], 
                                row['lon1']),
                      popup=row["address1"],
                      ).add_to(mapa)
    
    if((row["lat2"] == row["lat2"]) & (row["lon2"] == row["lon2"])):
        points.append((row["lat2"],row["lon2"]))
        folium.Marker(location=(row['lat2'], 
                                row['lon2']),
                      icon = folium.Icon(color="red",icon="info-sign"),
                      popup=row["address2"],
                      ).add_to(mapa)
        
    if(len(points)>0):
        folium.PolyLine(points
                        ,color="blue"
                        ,weight=2.5
                        ,popup= str(row["distancia_km"])+ " km"
                        ,opacity=1,
                        ).add_to(mapa)

display(mapa)